# The purpose of this notebook is to compute the 1D Power Spectrum from the Ly-$\alpha$ forest. 

In [2]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import fitsio 
import sys
from glob import glob
sys.path.append('/home/rodrigo/Documentos/env_pru/empca/')
%matplotlib inline
%pylab inline 

Populating the interactive namespace from numpy and matplotlib


In [3]:
def resample_flux(xout, x, flux, ivar=None, extrapolate=False):
    if ivar is None:
        return _unweighted_resample(xout, x, flux, extrapolate=extrapolate)
    else:
        if extrapolate :
            raise ValueError("Cannot extrapolate ivar. Either set ivar=None and extrapolate=True or the opposite")
        a = _unweighted_resample(xout, x, flux*ivar, extrapolate=False)
        b = _unweighted_resample(xout, x, ivar, extrapolate=False)
        mask = (b>0)
        outflux = np.zeros(a.shape)
        outflux[mask] = a[mask] / b[mask]
        dx = np.gradient(x)
        dxout = np.gradient(xout)
        outivar = _unweighted_resample(xout, x, ivar/dx)*dxout
        
        return outflux, outivar

def _unweighted_resample(output_x,input_x,input_flux_density, extrapolate=False) :
    
    # shorter names
    ix=input_x
    iy=input_flux_density
    ox=output_x

    # boundary of output bins
    bins=np.zeros(ox.size+1)
    bins[1:-1]=(ox[:-1]+ox[1:])/2.
    bins[0]=1.5*ox[0]-0.5*ox[1]     # = ox[0]-(ox[1]-ox[0])/2
    bins[-1]=1.5*ox[-1]-0.5*ox[-2]  # = ox[-1]+(ox[-1]-ox[-2])/2
    
    tx=bins.copy()
    if not extrapolate :
        # note we have to keep the array sorted here because we are going to use it for interpolation
        ix = np.append( 2*ix[0]-ix[1] , ix)
        iy = np.append(0.,iy)
        ix = np.append(ix, 2*ix[-1]-ix[-2])
        iy = np.append(iy, 0.)

    ty=np.interp(tx,ix,iy)
    
    #  add input nodes which are inside the node array
    k=np.where((ix>=tx[0])&(ix<=tx[-1]))[0]
    if k.size :
        tx=np.append(tx,ix[k])
        ty=np.append(ty,iy[k])
        
    # sort this node array
    p = tx.argsort()
    tx=tx[p]
    ty=ty[p]
    
    trapeze_integrals=(ty[1:]+ty[:-1])*(tx[1:]-tx[:-1])/2.
    
    trapeze_centers=(tx[1:]+tx[:-1])/2.
    binsize = bins[1:]-bins[:-1]

    if np.any(binsize<=0)  :
        raise ValueError("Zero or negative bin size")
    
    return np.histogram(trapeze_centers, bins=bins, weights=trapeze_integrals)[0] / binsize

In [4]:
fdir = '/home/rodrigo/Documentos/maestria/ultimo_año/datos_boss_dr9/BOSSLyaDR9_cat.fits'
DM = fitsio.FITS(fdir)
fm = fitsio.read(fdir,'BOSSLyaDR9_cat')
fm.dtype.descr

[('SDSS_NAME', '<U18'),
 ('RA', '>f4'),
 ('DEC', '>f4'),
 ('THING_ID', '>i4'),
 ('PLATE', '>i4'),
 ('MJD', '>i4'),
 ('FIBERID', '>i4'),
 ('Z_VI', '>f4'),
 ('Z_PIPE', '>f4'),
 ('SNR', '>f4'),
 ('SNR_LYAF', '>f4'),
 ('CHISQ_CONT', '>f4'),
 ('CONT_FLAG', '>i4'),
 ('CONT_TEMPLATE', '<U8'),
 ('Z_DLA', '>f4'),
 ('LOG_NHI', '>f4')]

In [5]:
mask = (fm['Z_VI'] > 2.1) & (fm['Z_VI'] < 4.0)
plate = fm['PLATE'][mask]
mjd = fm['MJD'][mask]
fiber = fm['FIBERID'][mask]
z_vi = fm['Z_VI'][mask]
pixels = []#np.zeros(plate.size)
for i in range(plate.size):
    pixels.append('BOSSLyaDR9_spectra/{}/speclya-{}-{}-{}.fits'.format(fm['PLATE'][i],fm['PLATE'][i],fm['MJD'][i],str(fm['FIBERID'][i]).zfill(4)))
    

In [6]:
z_vi.size

53809

In [7]:
pixels = np.array(pixels)

In [8]:
pixels

array(['BOSSLyaDR9_spectra/4216/speclya-4216-55477-0312.fits',
       'BOSSLyaDR9_spectra/4296/speclya-4296-55499-0364.fits',
       'BOSSLyaDR9_spectra/4277/speclya-4277-55506-0896.fits', ...,
       'BOSSLyaDR9_spectra/4212/speclya-4212-55447-0932.fits',
       'BOSSLyaDR9_spectra/4213/speclya-4213-55449-0570.fits',
       'BOSSLyaDR9_spectra/4282/speclya-4282-55507-0216.fits'],
      dtype='<U52')

I must match redshifts of the BOSSLyaDR9_cat.dat file with each speclya file. For this, I will try to build the path where the file is located.

In [9]:
def speclya(dirfile):
    fm = fitsio.read(dirfile,'BOSSLyaDR9_cat')
    mask = (fm['Z_VI'] > 2.1) & (fm['Z_VI'] < 4.0)
    plate = fm['PLATE'][mask]
    mjd = fm['MJD'][mask]
    fiber = fm['FIBERID'][mask]
    #RA = fm['RA']
    #DEC = fm['DEC']
    z_vi = fm['Z_VI'][mask]
    pixels = []
    for i in range(plate.size):
        pixels.append('BOSSLyaDR9_spectra/{}/speclya-{}-{}-{}.fits'.format(fm['PLATE'][i],fm['PLATE'][i],fm['MJD'][i],str(fm['FIBERID'][i]).zfill(4)))
    pixels = np.array(pixels)
    return z_vi, pixels

In [48]:
def read_pixels(dirfile):
    z, pixels = speclya(dirfile)
    fluxs = np.zeros((z.size,4630))
    ivars = np.zeros([z.size,4630])
    wave_rf = np.zeros([z.size,4630])
    for i in pixels[:1]:
        specfile = '/home/rodrigo/Documentos/maestria/ultimo_año/datos_boss_dr9/{}'.format(i)
        sp = fitsio.read(specfile)
        wave = 10**(sp['LOGLAM'])
        fluxs[i] = sp['FLUX']
        ivars[i] = sp['IVAR']
        wave_rf[i] = wave/(1 + z[i])
    fluxs = np.vstack(fluxs)
    ivars = np.vstack(ivars)
    wave_rf = np.vstack(wave_rf)
    return wave_rf, fluxs, ivars

In [49]:
spec = read_pixels(fdir)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [19]:
spec[0].size

4618

In [27]:
spec2 = np.asarray(spec)
spec2[0].shape

(4618,)

In [17]:
spec = np.array(spec)
spec

array([array([10.649687 ,  7.769591 , 13.353377 , ...,  2.2280293,  5.013948 ,
        5.1487546], dtype=float32),
       array([ 3.472002  ,  7.700982  ,  0.10725138, ...,  3.0779703 ,
        4.5454984 , -1.2245258 ], dtype=float32),
       array([-3.703024 ,  3.1218946,  1.4413437, ..., -2.5238564,  8.197158 ,
       -1.6799719], dtype=float32),
       array([11.769529 ,  8.239067 ,  4.7086263, ...,  0.6653994,  3.5595586,
        2.883961 ], dtype=float32),
       array([ 9.848478  ,  3.9781451 , -1.8923675 , ...,  0.25246808,
        3.727416  ,  5.360116  ], dtype=float32)], dtype=object)

In [14]:
specc = np.vstack(spec)

ValueError: all the input array dimensions except for the concatenation axis must match exactly